In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os
os.chdir("/home/yw699/codes/LLM-Hallu")
sys.path.append(os.path.abspath("src"))

## 读取配置文件

In [3]:
import yaml
with open("configs/example.yaml", "r") as file:
    config = yaml.safe_load(file)

## Dataset加载与格式化

In [4]:
from dataset import Dataset
dataset_loader = Dataset(config)
train_dataset, validation_dataset = dataset_loader.load_data()

print(validation_dataset[:1])  

[{'question': 'What is the smallest country in the world that is at least one square mile in area?', 'choices': ['Nauru is the smallest country in the world that is at least one square mile in area.', 'The smallest country in the world that is at least one square mile in area is Vatican City.', 'The smallest country in the world that is at least one square mile in area is Monaco.', 'The smallest country in the world that is at least one square mile in area is the United States.'], 'answers': {'text': [1, 0, 0, 0]}, 'id': '178297608846220991415708932806414338553'}]


## Prompt Engineer

In [5]:
from prompt_engineer import PromptGenerator
promptgenerator = PromptGenerator(config,validation_dataset)

In [6]:
prompt = promptgenerator.generate_prompt_by_id(1)
len(promptgenerator.get_unused_indices())
print(prompt)

USER: People in the US have longer paid vacations than people in which Western countries?
ASSISTANT:


In [7]:
prompts =promptgenerator.generate_prompts_by_count(5)
len(promptgenerator.get_unused_indices())

811

## model

In [8]:
from models import HuggingfaceModel
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"

In [14]:
huggingface_model = HuggingfaceModel(config)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [15]:
#temperature=1.0,return_full=False
output_text = huggingface_model.predict(prompt)
print("Generated Text > \n", output_text)

Generated Text > 
 ('Australia, Japan and New Zealand.', [-3.538668155670166, -1.0065431594848633, -4.511222839355469, -1.857227087020874, -1.7889091968536377, 0.0, -0.46492892503738403])


In [16]:
import math

user_tag = "USER:"
assistant_tag = "ASSISTANT:"
input = "What is the capital of France?"
v = huggingface_model.get_p_true(input,"Washington")
probability = math.exp(v)
print(f"Probability: {probability:.4f}")


v = huggingface_model.get_p_true(input,"Paris")
probability = math.exp(v)
print(f"Probability: {probability:.4f}")


v = huggingface_model.get_p_true(input,"Beijing")
probability = math.exp(v)
print(f"Probability: {probability:.4f}")

v = huggingface_model.get_p_true(input,"\n")
probability = math.exp(v)
print(f"Probability: {probability:.4f}")

What is the capital of France? Washington
Probability: 0.0080
What is the capital of France? Paris
Probability: 0.2130
What is the capital of France? Beijing
Probability: 0.0314
What is the capital of France? 

Probability: 0.0092


In [19]:
input_text = prompts[1]

output_text,log = huggingface_model.predict(input_text)
print("<Generated Text>:", output_text)
print(log)

<Generated Text>: Well, I suppose that's entirely a matter of the girl's character, isn't it?
[-2.270982503890991, -0.018149960786104202, -2.8177857398986816, -2.002683401107788, -1.810556173324585, -2.939112424850464, 0.0, -4.383170127868652, -3.0253024101257324, 0.0, -0.13263241946697235, -3.235642671585083, -0.8067711591720581, -0.17473973333835602, 0.0, -3.7862918376922607, -2.093878746032715, -0.852859616279602, 0.0, 0.0, 0.0, 0.0]
